# Minesweeper: Knowledge Bases & Logical Inference 

By **Rohan Rele** (rsr132), **Aakash Raman** (abr103), **Alex Eng** (ame136), and **Adarsh Patel** (aap237)

This project was completed for Professor Wes Cowan's Fall 2019 offering of the CS 520: Intro to Artificial Intelligence course, taught at Rutgers University, New Brunswick.

## Baseline Algorithm 

We began the assignment by implementing the basic solver agent algorithm that relies solely on local inference and iterative deduction. 

### Implementation 

##### Class Specification

Throughout this assignment we maintained a very object-oriented approach. The baseline case includes two main components. The Agent class and the MineSweeper class. The Agent represents the "player" of the game while the MineSweeper class represents the game as it transitions through its different states. Below are the class details for the two. 

In [12]:
class agent:
    def __init__(self, game, order):
        # copy game into agent object
        self.game = deepcopy(game)
        # track player knowledge per cell: initially all hidden
        self.playerKnowledge = np.array([[HIDDEN] * self.game.dim for _ in range(self.game.dim)])

        # keep track of these throughout game to yield final mineFlagRate
        self.numFlaggedMines = 0
        self.numDetonatedMines = 0

        # these are the two metrics we can judge performance with
        self.mineFlagRate = 0
        self.totalSolveTime = 0
        self.logging = False

        self.order = order
        self.current_in_order = 0

        self.uncertaintyType = 'none'


As we can see, the agent's members include:

1. (Type: Minesweeper) A MineSweeper object, which specifies the current instance of the game that agent is playing on
2. (Type: Numpy Array 2d) An integer matrix that keeps track of what cells have been queried, their *value* (vicinity to a mine) and/or their "mine status"

And the rest are simple metrics used to track the agent's progress: 

3. (Type: Integer) Number of flagged mines 
4. (Type: Integer) Number of detonated mines
5. (Type: Integer) Ratio between mines and flags
6. (Type: Integer) Time taken 
7. (Type: Boolean) Logger 
8. (Type: ? ) Order ? 
9. (Type: Integer) Current order ? 
10. (Type: String) Uncertainty ? 


In [5]:
dirs = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (-1, 1), (1, -1)]

MINE = -6
DETONATED = -8
HIDDEN = -1
SAFE = 0

class MineSweeper:
    dim = 0
    num_mines = 0
    board = [[]]
    success = False

    def __init__(self, dim, num_mines):
        self.dim = dim

        if num_mines < 0 or num_mines > dim**2:
            print("Invalid number of mines specified --> set to 0")
            num_mines = 0
        self.num_mines = num_mines

        board = [[0 for _ in range(dim)] for _ in range(dim)]

        for n in range(num_mines):
            x = random.randint(0, dim-1)
            y = random.randint(0, dim-1)
            while board[x][y] == MINE:
                x = random.randint(0, dim-1)
                y = random.randint(0, dim-1)
            board[x][y] = MINE

        temp = np.array(board)

        coords = zip(*np.where(temp == MINE))
        for x, y in coords:
            for i, j in dirs:
                if 0 <= x + i < dim and 0 <= y + j < dim and temp[x+i][y+j] != MINE:
                    temp[x + i][y + j] += 1
                else:
                    continue

        self.board = temp.tolist()


The minesweeper class is even more straightforward and enables the following: 

(I)  . Specifies coordinates to easily iterate through local cells

(II) . Specifies a key for the board (mine, hidden, safe...etc.) 

(III). Places random mines throughout our data structure based on some params (dim = dimension, n = # of mines) 

(IV) . Updates data structure with clues (+= 1 to every surrounding cell of a mine) 

Its members are: 

1. (Type: Integer) Dimension 
2. (Type: Integer) Number of mines 
3. (Type: 2d list) Board 
4. (Type: Boolean) Whether or not the board was solved


##### Solver 

As is the baseline or "naive" approach to minesweeper, our solver algorithm begins with a random first move (i.e. revealing a cell). It then tries to deduce, based solely on the 8 cells adjacent to it, whether or not any of the 9 cells in question are definitely safe or definitely a mine. Clearly, the chosen cell does not need to be inferred, but the 8 cells adjacent can generally not be inferred on the first move *unless* the clue is 0. Mathematically, we can denote this as: $P(X_{i,j} = 0 | C) = (1-d)^9$ where $C$ is the board configuration and $d$ is the mine density (Note: we are assuming the first cell queried is *not* a corner or edge cell, but our implementation handles all cases).  

See *attached* <font color=blue>log.txt</font> for full breakdown of moves made by baseline agent. The following snippet illustrates the above methodology. 

#BASELINE SOLVER LOG (initial moves) 

Cell (8, 2) safely revealed with clue: 3.
	# safe neighbors: 0
	# mine neighbors: 0
	# hidden neighbors: 8
	# total neighbors: 8


Revealing cell (8, 2) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------

Cell (3, 5) safely revealed with clue: 1.
	# safe neighbors: 0
	# mine neighbors: 0
	# hidden neighbors: 8
	# total neighbors: 8


Revealing cell (3, 5) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------

BOOM! Mine detonated at (5, 5).


Revealing cell (5, 5) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------

Cell (3, 7) safely revealed with clue: 1.
	# safe neighbors: 0
	# mine neighbors: 0
	# hidden neighbors: 8
	# total neighbors: 8


Revealing cell (3, 7) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------

Cell (11, 7) safely revealed with clue: 0.
	# safe neighbors: 0
	# mine neighbors: 0
	# hidden neighbors: 8
	# total neighbors: 8


All neighbors of (11, 7) must be safe.

	Neighbor (11, 8) flagged as safe and enqueued for next visitation.

	Neighbor (11, 6) flagged as safe and enqueued for next visitation.

	Neighbor (12, 7) flagged as safe and enqueued for next visitation.

	Neighbor (10, 7) flagged as safe and enqueued for next visitation.

	Neighbor (12, 8) flagged as safe and enqueued for next visitation.

	Neighbor (10, 6) flagged as safe and enqueued for next visitation.

	Neighbor (10, 8) flagged as safe and enqueued for next visitation.

	Neighbor (12, 6) flagged as safe and enqueued for next visitation.

----------------------------------------

...


When enough of the board has been filled, the metrics of each cell start to be the main source of knowledge for the agent. At every cell $(x,y)$, the agent keeps track of $(x,y)$'s total neighbors, its safe neighbors, mine neighbors and hidden neighbors, which can be used for marking more cells as safe/unsafe. For example, if a cell has 8 total neighbors, 7 safe neighbors, 1 hidden neighbor and a clue of 1, the agent can determine with 100% certainty that the final neighbor is a mine. Once the agent can no longer make any reliable inferences, it returns to the last safe, visited cell which is stored in a queue. If the queue is empty, the agent returns to random selection

Below is a snippet from <font color=blue>log.txt</font> illustrating this more concretely. 

#BASELINE SOLVER LOG (intermediate moves) 

...

----------------------------------------

Cell (6, 5) safely revealed with clue: 4.
	# safe neighbors: 2
	# mine neighbors: 2
	# hidden neighbors: 4
	# total neighbors: 8


Revealing cell (6, 5) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing KB found that: All neighbors of (2, 5) must be safe.

		Neighbor (1, 6) flagged as safe and enqueued for next visitation.

----------------------------------------

Cell (1, 6) safely revealed with clue: 3.
	# safe neighbors: 4
	# mine neighbors: 1
	# hidden neighbors: 3
	# total neighbors: 8


Revealing cell (1, 6) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing KB found that: All neighbors of (0, 5) must be mines.

		Neighbor (0, 6) flagged as a mine.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------

Cell (12, 4) safely revealed with clue: 5.
	# safe neighbors: 0
	# mine neighbors: 0
	# hidden neighbors: 8
	# total neighbors: 8


Revealing cell (12, 4) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------
...

*All iterations of <font color=blue>log.txt</font> were from the following game (refer to key for integer meanings)* 

<table>
    <tr>
        <td>
<figure>
    <img src='baselineTrialRun_init_board.png' width="400" height="200" alt='missing' />
    <figcaption>Initial Board (Unhidden)</figcaption>
</figure>
        </td>
        <td>
<figure>
    <img src='baselineTrialRun_solved_board.png' width="400" height="200" alt='missing' />
    <figcaption>Solved Board</figcaption>
</figure>
        </td>
    </tr>
</table>

### Performance & Results 

// add data here about performance of baseline with different densities/dims 

// time complexity, worst case 

## Improved Algorithm 

We decided on an approach that relies not solely on local inference but evaluates all the clues given to evaluate the whole board at every iteration. We will refer to this as the **Linear Algebra Approach**. 

### Implementation 

##### Representation 

Our Linear Algebra Agent (*see <font color=blue>LinAlg.py</font>*) inherits all members & methods from our generic Agent class, and thus its object representation is nearly identical (except one extra metric for computational convenience); as is the board structure that it operates on. 

In [6]:
class lin_alg_agent(agent):
    def __init__(self, game, useMineCount, order):
        agent.__init__(self,game, order)
        self.useMineCount = useMineCount

##### Inference 

Rather than performing inference based on a restricted set of clues/information about adjacent data, the Linear Algebra appraoch does inference on the entire board, which effectively expands the knowledge base by a factor of $(dim-3)^2$. It then uses that expanded knowledge base to create a constraint satisfaction problem or, more conceretly, a system of linear equations that analytically describes what cells are *guaranteed* to be mines. 

This is illustrated below:

<table>
    <tr>
        <td>
<figure>
    <img src='ss1.png' width="150" height="650" alt='missing' />
    <figcaption>Example block of game board with variable labels</figcaption>
</figure>
        </td>
        <td>
<figure>
    <img src='ss2.png' width="300" height="150" alt='missing' />
    <figcaption>Corresponding System of Equations</figcaption>
</figure>
        </td>
  <td>
<figure>
    <img src='ss4.png' width="300" height="150" alt='missing' />
    <figcaption>Corresponding Matrix</figcaption>
</figure>
        </td>
    </tr>
</table>

*Images from https://massaioli.wordpress.com/2013/01/12/solving-minesweeper-with-matricies/comment-page-1/*

##### Decisions 

Decision making is fairly straightforward, once the matrix is row reduced, we can mark cells that correspond to reduced rows with a 0 in the final column vector as being safe and similarly reduced rows with value 1 in the final column vector as mine. Unreduced or 0 row vectors are disregarded. 

The Linear Algebra Agent's decision-making process is logged out in a snippet below (see <font color=blue>lin_alg_log.txt</font>):

...

----------------------------------------

Cell (2, 2) safely revealed with clue: 3.
	# safe neighbors: 4
	# mine neighbors: 1
	# hidden neighbors: 3
	# total neighbors: 8


Revealing cell (2, 2) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

game:
[0, 0, 0, 0]
[1, 1, 1, 1]
[-6, 2, 3, -6]
[2, -6, 3, -6]

knowledge:
[[ 0  0  0  0]
 [ 0  0  0  0]
 [-6  0  0 -6]
 [-1 -1 -1 -1]]

generated following row using (2,1): 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1.]

generated following row using (2,2): 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 2.]

generated following row using total mine count: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 2.]

information matrix:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 2.]]

rref'd matrix:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]

using row: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
deduced (3,0) to be safe via lin alg

using row: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.]

using row: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
deduced (3,3) to be a mine via lin alg

----------------------------------------

Cell (3, 0) safely revealed with clue: 2.
	# safe neighbors: 1
	# mine neighbors: 1
	# hidden neighbors: 1
	# total neighbors: 3


All neighbors of (3, 0) must be mines.

	Neighbor (3, 1) flagged as a mine.

Revealing cell (3, 0) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

game:
[0, 0, 0, 0]
[1, 1, 1, 1]
[-6, 2, 3, -6]
[2, -6, 3, -6]

knowledge:
[[ 0  0  0  0]
 [ 0  0  0  0]
 [-6  0  0 -6]
 [ 0 -6 -1 -6]]

generated following row using (2,1): 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]

generated following row using (2,2): 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]

generated following row using total mine count: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]

information matrix:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]

rref'd matrix:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

using row: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
deduced (3,2) to be safe via lin alg

using row: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

using row: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

----------------------------------------

Cell (3, 2) safely revealed with clue: 3.
	# safe neighbors: 2
	# mine neighbors: 3
	# hidden neighbors: 0
	# total neighbors: 5


All neighbors of (3, 2) are already revealed; nothing to infer.

----------------------------------------
...

*The game board that yielded the above log is displayed here:*

// insert image of game board

### Performance & Results 

// Efficiency 

### Improvements from the Baseline 

// add some rough figures here about how much this algorithm improved the baseline win rate

// then add comparison graphs between baseline and improved v 1.0 

## Improved Algorithm v 1.5

This is a modified version of the **Linear Algebra approach** that makes improvements to guessing. Any algorithm to solve Minesweeper will have to eventually make a random guess. The improvements outlined in this version of our approach attempts to address this problem and instead make an *educated guess* using a combination of brute force and probabilistic inference 

### Further Improvements from Baseline 

// rough data here about how much it improved after adding brute force

### Implementation 

The Brute Force Agent inherits all of the Linear Algebra Agent's (and thus, the Generic Agent's) members/functions. The object-oriented schema remains unchanged throughout this assignment. 

In [11]:
class brute_force_agent(lin_alg_agent):
    def __init__(self, game, useMineCount, order):
        lin_alg_agent.__init__(self,game,useMineCount,order)

The main advantage of adding brute force functionality is that upon failure of the Linear Algebra Agent's main method (i.e. solving the system of equations via matrix algebra), it no longer has to randomly guess what the next move will be. Instead, we iterate through all possible configurations (of relevant cells) and keep track of how many times a mine appeared in each cell. We then simply choose the cell with the least probability of having a mine. 

The methods that perform these computations are the configuration generation function : 

In [10]:
def get_configs(self, configCells, consistency_cells):
        configs = [(deepcopy(self.playerKnowledge), i, [])  for i in range(len(configCells))]
        out = []
        while configs:
            next_set = []
            for board, index, currentConfig in configs:
                current_board = deepcopy(board)
                current_config = deepcopy(currentConfig)
                dim = self.game.dim

                if index >= len(configCells) or (self.useMineCount and len(current_config) == self.game.num_mines-self.numFlaggedMines-self.numDetonatedMines):
                    if self.confirm_full_consistency(current_board, consistency_cells):
                        out.append(current_config)
                    continue
                cell = configCells[index]
                current_config.append(cell)
                x = cell[0]
                y = cell[1]
                current_board[x,y] = MINE
                valid = True
                for dx, dy in dirs:
                    nx = x + dx
                    ny = y + dy
                    if 0 <= nx < dim and 0 <= ny < dim and self.playerKnowledge[nx][ny] == SAFE and not self.confirm_consistency(current_board, nx, ny):
                        valid = False
                if valid:
                    for i in range(index + 1, len(configCells) + 1):
                        next_set.append((current_board,i,current_config))

            configs = next_set
        return out

and the probability calculating function: 

In [8]:
def probability_method(self):
        # print(self.playerKnowledge)
        dim = self.game.dim
        consistency_cells = []
        config_cells = list()
        for x in range(dim):
            for y in range(dim):
                if self.playerKnowledge[x][y] == SAFE:
                    numSafeNbrs, numMineNbrs, numHiddenNbrs, numTotalNbrs = self.getCellNeighborData(x, y)
                    if numHiddenNbrs > 0:
                        consistency_cells.append((x,y))
                        children = set(self.get_hidden_neighbors(x,y))
                        # print(children)
                        intersects = []
                        for i,s in enumerate(config_cells):
                            if len(children.intersection(s)) > 0:
                                intersects.append(i)
                        if len(intersects) > 0:
                            for i in intersects:
                                children.update(config_cells[i])
                            for i in intersects[::-1]:
                                del config_cells[i]
                        config_cells.append(children)
        #                 print(config_cells)
        # print(config_cells)

        if len(consistency_cells) == 0:
            return self.get_next_random(set())

        config_cells = [sorted(list(y), key = lambda x: x[0] * dim + x[1]) for y in config_cells]

        configs = []
        to_remove = []
        for i,s in enumerate(config_cells):
            if len(s) > 20:
                to_remove.append(i)
                # print("len(s)={}, ignoring".format(len(s)))
                continue
            start_search_time = time.time()
            configs.append(self.get_configs(s, consistency_cells))
            if time.time() - start_search_time > 10:
                print("len(s)={}, took {} seconds to compute".format(len(s), round(time.time() - start_search_time,2)))
        # print([len(s) for s in config_cells])
        for i in to_remove[::-1]:
            del config_cells[i]
        # print([len(s) for s in config_cells])
        # print(len(config_cells))
        # print(len(configs))
        if len(configs) == 0:
            return self.get_next_random(set())


        min_mine_count = sum([ 0 if len(s) == 0 else min([len(config) for config in s]) for s in configs])
        for s in configs:
            min_for_set =  0 if len(s) == 0 else min([len(config) for config in s])
            to_remove = []
            for i,config in enumerate(s):
                if len(config) + min_mine_count - min_for_set > self.game.num_mines-self.numFlaggedMines-self.numDetonatedMines:
                    to_remove.append(i)
            for i in to_remove[::-1]:
                del s[i]

        # print("found configs:")
        probabilities = dict()
        for i,s in enumerate(configs):
            if len(s) > 0:
                counts = {x:0 for x in config_cells[i]}
                for config in s:
                    for coordinates in config:
                        counts[coordinates] += 1
                for k , v in counts.items():
                    probabilities[k] = v / len(s)
            else:
                for cell in config_cells[i]:
                    probabilities[cell] = 0

        best_cell = None
        best_probability = len(configs)

        for cell, probability in probabilities.items():
            if probability <= best_probability:
                best_cell = cell
                best_probability = probability

        other_cells = self.numHiddenCells() - len(config_cells)
        mines_left = self.game.num_mines-self.numFlaggedMines-self.numDetonatedMines
        max_mines_in_hidden_cells_not_in_config_Cells = mines_left - min_mine_count
        other_cell_max_probability = 1 if other_cells == 0 else max_mines_in_hidden_cells_not_in_config_Cells / other_cells

        if best_probability < other_cell_max_probability:
            assert best_cell is not None
            return best_cell
        else:
            to_exclude = []
            for l in config_cells:
                to_exclude.extend(l)
            return self.get_next_random(set(to_exclude))


### Results 

## Bonus - Dealing with Uncertainty 

### Accurate Information (non-sequentially) 

### Flawed Knowledge-Base (underestimate)

### Flawed Knowledge-Base (overestimate) 